In [44]:
import pandas as pd
import numpy as np
import random
from datetime import timedelta, datetime

In [45]:
random.seed(174)

# Values for data synthesis

In [46]:
cities = [
    "New York", "Los Angeles", "Chicago", "Houston", "Phoenix",
    "Philadelphia", "San Antonio", "San Diego", "Dallas", "San Jose",
    "Austin", "Jacksonville", "Fort Worth", "Columbus", "Charlotte",
    "San Francisco", "Indianapolis", "Seattle", "Denver", "Washington",
    "Boston", "El Paso", "Nashville", "Detroit", "Oklahoma City",
    "Portland", "Las Vegas", "Memphis", "Louisville", "Baltimore",
    "Milwaukee", "Albuquerque", "Tucson", "Fresno", "Mesa",
    "Sacramento", "Atlanta", "Kansas City", "Colorado Springs", "Miami",
    "Raleigh", "Omaha", "Long Beach", "Virginia Beach", "Oakland",
    "Minneapolis", "Tulsa", "Arlington", "Tampa", "New Orleans",
    "London", "Paris", "Tokyo", "Beijing", "Moscow",
    "Sydney", "Dubai", "Mexico City", "Sao Paulo", "Mumbai",
    "Cairo", "Istanbul", "Lagos", "Buenos Aires", "Seoul",
    "Bangkok", "Kolkata", "Tehran", "Berlin", "Bogota",
    "Lima", "Jakarta", "Riyadh", "Madrid", "Rome"
]

In [47]:
keywords = [
    "artificial intelligence", "climate change", "sustainable development",
    "quantum computing", "genetic engineering", "machine learning", "cybersecurity",
    "public health", "renewable energy", "data analytics", "blockchain", "nanotechnology",
    "neural networks", "ecosystem services", "cancer research", "autonomous vehicles",
    "internet of things", "big data", "vaccine development", "bioinformatics",
    "virtual reality", "augmented reality", "deep learning", "biodiversity",
    "mental health", "smart cities", "robotics", "3D printing", "cloud computing",
    "CRISPR", "agritech", "clean technology", "material science", "photovoltaics",
    "drug discovery", "astrophysics", "oceanography", "glaciology", "sociology",
    "economics", "political science", "urban planning", "microbiology", "quantum mechanics",
    "biochemistry", "particle physics", "organic chemistry", "computational biology",
    "environmental justice", "sustainable agriculture", "water scarcity", "air pollution",
    "soil degradation", "renewable resources", "energy storage", "machine ethics",
    "digital humanities", "biomedical engineering", "forensic science", "epidemiology",
    "neuroscience", "cognitive science", "psychology", "anthropology", "linguistics",
    "education technology", "space exploration", "conservation biology", "green chemistry", 
    "industrial automation", "wearable technology", "nutrition science",
    # mandatory keywords
    "data management", 
    "indexing", 
    "data modeling", 
    "big data", 
    "data processing", 
    "data storage",
    "data querying"
]

first_names = [
    "Alex", "Jamie", "Casey", "Morgan", "Taylor", "Jordan", "Riley", "Cameron", "Skyler", "Quinn", 
    "Pat", "Drew", "Sam", "Chris", "Robin", "Lee", "Dana", "Kelly", "Alexis", "Leslie"
]

last_names = [
    "Smith", "Johnson", "Williams", "Brown", "Jones", "Garcia", "Miller", "Davis", "Rodriguez", 
    "Martinez", "Wilson", "Anderson", "Thomas", "Taylor", "Moore", "Jackson", "Martin", "Lee", "Perez", "Thompson"
]

# Functions & Load data #

In [48]:
def read_custom_file(file_path, low_memory=False):
    df = pd.read_csv(file_path, sep=';', header=None, low_memory=low_memory)
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])
    return df

In [49]:
path = 'C:\\Users\\Dima\\SDM\\SDM_Project\\joined'

proceeding_df = pd.read_csv(f'{path}\\proceedings.csv', low_memory=False).sample(3250)
article_df = pd.read_csv(f'{path}\\article.csv', low_memory=False).sample(3250)
book_df = pd.read_csv(f'{path}\\book.csv', low_memory=False).sample(3250)

journal_df = read_custom_file(f'{path}\\journal.csv')
authors_df = read_custom_file(f'{path}\\author.csv')

In [50]:
proceeding_df

,Unnamed: 0,proceedings:ID,address:string,author:string[],booktitle:string,cite:string[],cite-label:string[],editor:string[],editor-orcid:string[],ee:string[],...,publtype:string,school:string,series:string[],series-href:string[],sub:string,sup:string[],title:string,url:string,volume:string,year:int
54907,54907,3404426,NaN,NaN,PST,NaN,NaN,NaN,NaN,https://ieeexplore.ieee.org/xpl/conhome/789923...,...,NaN,NaN,NaN,NaN,NaN,NaN,"14th Annual Conference on Privacy, Security an...",db/conf/pst/pst2016.html,NaN,2016
471,471,123880,NaN,NaN,CN,NaN,NaN,Andrzej Kwiecien|Piotr Gaj|Piotr Stera,NaN,https://doi.org/10.1007/978-3-642-02671-3,...,NaN,NaN,Communications in Computer and Information Sci...,db/series/ccis/index.html,NaN,NaN,"Computer Networks - 16th Conference, CN 2009, ...",db/conf/cn/cn2009.html,39,2009
32328,32328,2131749,NaN,NaN,Bytecode@ETAPS,NaN,NaN,Fausto Spoto|Marieke Huisman,NaN,https://www.sciencedirect.com/journal/electron...,...,NaN,NaN,Electronic Notes in Theoretical Computer Science,db/series/entcs/index.html,NaN,NaN,Proceedings of the Second Workshop on Bytecode...,db/conf/bytecode/bytecode2007.html,190,2007
43581,43581,2773939,NaN,NaN,CIA,NaN,NaN,Franco Zambonelli|Matthias Klusch,NaN,https://doi.org/10.1007/3-540-44799-7,...,NaN,NaN,Lecture Notes in Computer Science,db/series/lncs/index.html,NaN,NaN,"Cooperative Information Agents V, 5th Internat...",db/conf/cia/cia2001.html,2182,2001
24261,24261,1631030,NaN,NaN,ECCE,NaN,NaN,Annamaria Recupero|Oronzo Parlangeli|Patrizia ...,NaN,https://doi.org/10.1145/3452853,...,NaN,NaN,NaN,NaN,NaN,NaN,ECCE 2021: European Conference on Cognitive Er...,db/conf/ecce/ecce2021.html,NaN,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13755,13755,873059,NaN,NaN,ANALCO,NaN,NaN,Daniel Panario|Philippe Flajolet,NaN,https://doi.org/10.1137/1.9781611973013,...,NaN,NaN,NaN,NaN,NaN,NaN,Proceedings of the Eighth Workshop on Analytic...,db/conf/analco/analco2011.html,NaN,2011
23200,23200,1570215,NaN,NaN,ICC Workshops,NaN,NaN,NaN,NaN,https://ieeexplore.ieee.org/xpl/conhome/687805...,...,NaN,NaN,NaN,NaN,NaN,NaN,IEEE International Conference on Communication...,db/conf/icc/icc2014w.html,NaN,2014
28200,28200,1891906,NaN,NaN,DARS,NaN,NaN,Hajime Asama|Isao Endo|Tamio Arai|Toshio Fukuda,NaN,https://doi.org/10.1007/978-4-431-68275-2,...,NaN,NaN,NaN,NaN,NaN,NaN,"Distributed Autonomous Robotic Systems, Procee...",db/conf/dars/dars1994.html,NaN,1994
39640,39640,2531652,NaN,NaN,Workshop on Exception Handling,NaN,NaN,NaN,NaN,https://doi.org/10.1145/1454268,...,NaN,NaN,NaN,NaN,NaN,NaN,Proceedings of the 4th International Workshop ...,db/conf/sigsoft/weh2008.html,NaN,2008


In [51]:
def generate_city(): #For Edition Nodes - Synthetic info
    return np.random.choice(cities)
    
def generate_birth_date_vectorized(start='1945-01-01', end='1989-01-01', size=1):  #For Author Nodes - Synthetic info
    # Convert start and end dates to timestamps
    start_u = datetime.strptime(start, '%Y-%m-%d').timestamp()
    end_u = datetime.strptime(end, '%Y-%m-%d').timestamp()
    
    # Generate random dates
    random_dates_u = np.random.randint(start_u, end_u, size)
    random_dates = [datetime.utcfromtimestamp(date).strftime('%Y-%m-%d') for date in random_dates_u]
    
    return random_dates if size > 1 else random_dates[0]

def generate_proceeding_type():
    return np.random.choice(['WORKSHOP', 'CONFERENCE'], p=[0.4, 0.6])

def select_random_keywords(keywords=keywords, min_keywords=4, max_keywords=8):
    num_to_select = random.randint(min_keywords, max_keywords)
    selected_keywords = random.sample(keywords, num_to_select)
    return selected_keywords

In [52]:
# Helpers
def filter_rows_by_values(df, column_name, values_set):
    return df[df[column_name].isin(values_set)]

def make_mapping(df, key_column, value_column):
    return pd.Series(df[key_column], index=df[value_column]).to_dict()

def create_df_from_values_with_keys(values, value_col_name, key_col_name, key_pattern='ID-{:06d}'):
    df = pd.DataFrame({
        key_col_name: [key_pattern.format(i+1) for i in range(len(values))],
        value_col_name: values
    })
    return df

def collect_values_to_set(df, column_name):
    return set(df[column_name].explode().dropna().unique())


In [53]:
def add_citations_column(df):
    sorted_df = df.sort_values(by='year').copy()
    citations_dict = {index: [] for index in sorted_df.index}
    
    # Pre-compute the valid citation ids for each year to avoid doing it in the loop
    valid_citations_per_year = {year: sorted_df[sorted_df['year'] < year]['paper_id'].tolist() for year in sorted_df['year'].unique()}
    
    for index, row in sorted_df.iterrows():
        valid_citations = valid_citations_per_year[row['year']]
        
        if valid_citations:
            num_ids_to_select = min(np.random.randint(3, 7), len(valid_citations))
            selected_ids = np.random.choice(valid_citations, size=num_ids_to_select, replace=False)
            citations_dict[index] = selected_ids.tolist()
    
    sorted_df['citation_id'] = pd.Series(citations_dict).apply(lambda ids: ids if isinstance(ids, list) else pd.NA)
    return sorted_df[('citation_id')]


def choose_reviewers(authors, reviewers_set):
    authors_set = set(authors)
    all_reviewers = list(reviewers_set - authors_set)
    num_reviewers = np.random.choice([1, 2, 3, 4, 5], p=[0.05, 0.15, 0.6, 0.15, 0.05])
    
    if len(all_reviewers) >= num_reviewers:
        return np.random.choice(all_reviewers, size=num_reviewers, replace=False).tolist()
    
    chosen_reviewers = all_reviewers
    remaining_needed = num_reviewers - len(chosen_reviewers)
    
    if remaining_needed > 0:
        fill_ins = np.random.choice(list(reviewers_set - authors_set), size=remaining_needed, replace=False).tolist()
        chosen_reviewers.extend(fill_ins)
    
    return chosen_reviewers

def add_columns_from_authors(df, authors_col_name, reviewers_set):
    df['core_author'] = df[authors_col_name].str[0]
    df['co_author'] = df[authors_col_name].apply(lambda authors: authors[1:] if len(authors) > 1 else pd.NA)
    
    # Assuming 'authors' are lists, we can avoid an apply by directly using a list comprehension
    df['reviewer'] = [choose_reviewers(authors, reviewers_set) for authors in df[authors_col_name]]
    
    return df


In [54]:
def preprocess_authors(df):
    #birth_dates = generate_birth_date_vectorized(size=len(df))
    
    df = df.dropna(subset=[':ID', 'author:string']).drop_duplicates(subset=[':ID'], keep='first')
    
    df['id'] = df[':ID'].astype(str)
    df['full_name'] = df['author:string']
    #df['birth_date'] = birth_dates
    
    return df[['id', 'full_name']]

def preprocess_journals(df):
    df = df.loc[:, [':ID', 'journal:string']].drop_duplicates(subset=[':ID'], keep='first').dropna()
    df.rename(columns={':ID': 'id', 'journal:string': 'name'}, inplace=True)
    
    return df

def preprocess_books(df):
    filtered_df = df[(df['author:string[]'].str.len() > 2) & (df['title:string'].str.len() > 2)]
    
    return (
        filtered_df.assign(
            abstract='Abstract',
            paper_id='PPR-' + filtered_df['book:ID'].astype(str),
            authors=filtered_df['author:string[]'].apply(lambda x: x.split('|') if pd.notnull(x) else pd.NA),
            title=filtered_df['title:string'],
            year=filtered_df['year:int']
        )
        .loc[:, ['paper_id', 'authors', 'title', 'abstract', 'year']]
        .dropna(subset=['authors', 'year'])
    )

def preprocess_proceedings(df):
    # Fill missing values
    df['editor:string[]'] = df['editor:string[]'].fillna('')
    df['publisher:string[]'] = df['publisher:string[]'].fillna('')
    
    # Filter rows based on content length criteria
    filtered_df = df[(df['editor:string[]'].str.len() > 2) & (df['publisher:string[]'].str.len() > 2)].copy()
    
    # Assuming 'create_df_from_values_with_keys' returns a DataFrame with unique values and an ID column
    unique_proceeding_df = create_df_from_values_with_keys(filtered_df['publisher:string[]'].unique(), 'proceeding_name', 'proceeding_id', key_pattern='PR-{:06d}')
    proceeding_name_to_id_map = dict(zip(unique_proceeding_df['proceeding_name'], unique_proceeding_df['proceeding_id']))
    
    # Map 'publisher:string[]' to 'proceeding_id'
    filtered_df.loc[:, 'proceeding_id'] = filtered_df['publisher:string[]'].map(proceeding_name_to_id_map)
    
    # Extract the first editor as chairman
    filtered_df.loc[:, 'chairman'] = filtered_df['editor:string[]'].apply(lambda x: x.split('|')[0] if x else pd.NA)
    
    unique_chairman_df = create_df_from_values_with_keys(filtered_df['chairman'].unique(), 'person_name', 'person_id', key_pattern='CH-{:06d}')
    chairman_name_to_id_map = dict(zip(unique_chairman_df['person_name'], unique_chairman_df['person_id']))
    
    # Map 'chairman' to 'person_id'
    filtered_df.loc[:, 'chairman'] = filtered_df['chairman'].map(chairman_name_to_id_map)
    
    # Additional transformations
    filtered_df.loc[:, 'edition_id'] = filtered_df['proceeding_id'].astype(str) + '-' + filtered_df['volume:string'].astype(str)
    
    filtered_df = filtered_df.assign(
        name=filtered_df['publisher:string[]'],
        edition_value=filtered_df['volume:string'],
        edition_date=filtered_df['mdate:date'],
        edition_city=[generate_city() for _ in range(len(filtered_df))]
    ).dropna(subset=['edition_value']).drop_duplicates(subset=['edition_id'])

    # Select and rename columns as necessary
    final_df = filtered_df.loc[:, ['proceeding_id', 'chairman', 'name', 'edition_date', 'edition_id', 'edition_city']]
    
    return final_df, unique_proceeding_df, unique_chairman_df

def preprocess_articles(df):
    filtered_df = df[(df['author:string[]'].str.len() > 2) & (df['title:string'].str.len() > 2)]
    
    return (
        filtered_df.assign(
            paper_id=filtered_df['article:ID'].astype(str),
            authors=filtered_df['author:string[]'].apply(lambda x: x.split('|') if pd.notnull(x) else pd.NA),
            journal=filtered_df['journal:string'],
            date=filtered_df['mdate:date'],
            title=filtered_df['title:string'],
            volume=filtered_df['volume:string'],
            year=filtered_df['year:int'].apply(lambda x: int(x) if pd.notnull(x) else pd.NA)
        )
        .dropna(subset=['authors', 'journal', 'volume'])
        .loc[:, ['paper_id', 'authors', 'journal', 'date', 'title', 'volume', 'year']]
    )


# Cleaning and filling #

In [55]:
authors_prep_df = preprocess_authors(authors_df)

In [56]:
authors_prep_df

,id,full_name
1,10597993,Manish Singh 0001
2,10597994,Tien-Tsin Wong
3,10597995,Sylvia C. Pont
4,10597996,Daniel C. Alexander
5,10597997,Kazuhiro Fukui
...,...,...
3556892,14154884,Hak Chuah Sim
3556893,14154885,Holger Schackert
3556894,14154886,María Paz Casanova
3556895,14154887,Hendrikus Smid


In [57]:
journals_prep_df = preprocess_journals(journal_df)

In [58]:
journals_prep_df

,id,name
1,14154889,EAI Endorsed Trans. Ubiquitous Environ.
2,14154890,SIGMOD Record
3,14154891,SIGMOD Rec.
4,14154892,World Wide Web
5,14154893,Int. J. Learn. Technol.
...,...,...
2006,14156894,"Forschungsberichte, TU Munich"
2007,14156895,"ETH Zurich, Department of Computer Science / T..."
2008,14156896,Hydrology and Earth System Sciences
2009,14156897,"Technical Rep. UKSC 0060, IBM United Kingdom S..."


In [59]:
author_to_id_map = dict(zip(authors_prep_df['full_name'], authors_prep_df['id']))
journal_to_id_map = dict(zip(journals_prep_df['name'], journals_prep_df['id']))

In [60]:
papers_from_proceeding_df = preprocess_books(book_df)
papers_from_journal_df = preprocess_articles(article_df)
prep_proceedings_df, proceeding_nodes_df, chairman_nodes_df = preprocess_proceedings(proceeding_df)

In [61]:
papers_from_proceeding_df['authors'] = papers_from_proceeding_df['authors'].apply(lambda x: [author_to_id_map.get(a) for a in x] if x is not pd.NA else pd.NA)
reviewers_set = set(papers_from_proceeding_df['authors'].dropna().explode().unique())
papers_from_proceeding_df = add_columns_from_authors(papers_from_proceeding_df, 'authors', reviewers_set)
papers_from_proceeding_df['citation_id'] = add_citations_column(papers_from_proceeding_df)
papers_from_proceeding_df['keywords'] = [select_random_keywords() for _ in range(len(papers_from_proceeding_df))]

# Prepare the final DataFrame for proceedings
proceeding_prep_df = papers_from_proceeding_df[['paper_id', 'title', 'abstract', 'core_author', 'co_author', 'citation_id', 'keywords']]

In [62]:
papers_from_proceeding_df

,paper_id,authors,title,abstract,year,core_author,co_author,reviewer,citation_id,keywords
2309,PPR-39712,[10612435],Laboratory Experiments in Information Retrieva...,Abstract,2018,10612435,<NA>,"[14008098, 13244646, 10632533]","[PPR-38537, PPR-33029, PPR-10462918, PPR-10452...","[digital humanities, quantum computing, anthro..."
15436,PPR-10458720,[12885908],High-temperature-superconductor thin films at ...,Abstract,1999,12885908,<NA>,"[10629480, 10925386]","[PPR-10429433, PPR-10461336, PPR-10447798, PPR...","[cloud computing, agritech, ecosystem services..."
4088,PPR-51847,"[10654122, 10654121]",From scientific instrument to industrial machi...,Abstract,2012,10654122,[10654121],"[10608939, 10638459]","[PPR-37283, PPR-10462290, PPR-10462075, PPR-10...","[computational biology, data management, nanot..."
7580,PPR-10437867,"[10640381, 10631504]",Reasoning with Logic Programming,Abstract,1996,10640381,[10631504],"[10949635, 13588402, 12505467]","[PPR-10456933, PPR-10458697, PPR-10459718, PPR...","[green chemistry, cancer research, wearable te..."
9199,PPR-10446776,[12567759],Variation Based Dense 3D Reconstruction - Appl...,Abstract,2016,12567759,<NA>,"[10614279, 10600647, 11834161]","[PPR-10459812, PPR-51896, PPR-10457506, PPR-33...","[digital humanities, anthropology, drug discov..."
...,...,...,...,...,...,...,...,...,...,...
13151,PPR-10456160,"[10641232, 10600395, 11027237]",Algorithms.,Abstract,2008,10641232,"[10600395, 11027237]","[12274483, 12208937, 10785474]","[PPR-10454987, PPR-10461523, PPR-10456927]","[space exploration, computational biology, nut..."
18075,PPR-10461667,[11720125],Theorie et pratique du workflow - des processu...,Abstract,1997,11720125,<NA>,"[13875237, 10667556, 13995519]","[PPR-10441964, PPR-10461431, PPR-10461458, PPR...","[big data, deep learning, data modeling, econo..."
5808,PPR-10430360,[14119273],Contribution à l'analyse non linéaire.,Abstract,1976,14119273,<NA>,"[10623261, 11881555, 10849310]","[PPR-10452873, PPR-10429418, PPR-10430982, PPR...","[internet of things, augmented reality, data s..."
15330,PPR-10458605,"[10634933, 10641933]",Einführung in die funktionale Programmierung.,Abstract,1992,10634933,[10641933],"[11368590, 10636068, 12390700]","[PPR-10460319, PPR-10460820, PPR-10443697, PPR...","[machine ethics, forensic science, astrophysic..."


In [63]:
papers_from_proceeding_authors_set = collect_values_to_set(papers_from_proceeding_df, 'authors').intersection(set(authors_prep_df['full_name']))
papers_from_journal_authors_set = collect_values_to_set(papers_from_journal_df, 'authors').intersection(set(authors_prep_df['full_name']))
all_authors = papers_from_proceeding_authors_set.union(papers_from_journal_authors_set)

papers_from_proceeding_df = add_columns_from_authors(papers_from_proceeding_df, 'authors', all_authors).drop('authors', axis=1)
papers_from_journal_df = add_columns_from_authors(papers_from_journal_df, 'authors', all_authors).drop('authors', axis=1)

# limit authors df by all_authors
author_nodes_df = authors_prep_df[authors_prep_df['full_name'].isin(all_authors)]

In [64]:
def assign_event_type(df, id_column):
    unique_ids = df[id_column].unique()
    event_types = np.random.choice(['workshop', 'conference'], size=len(unique_ids))
    id_to_event_type = dict(zip(unique_ids, event_types))
    return df[id_column].map(id_to_event_type)

values = papers_from_proceeding_df['paper_id'].unique().tolist()
prep_proceedings_df['paper_id'] = prep_proceedings_df.apply(lambda _: list(np.random.choice(values, size=np.random.randint(2, 7), replace=False)), axis=1)
prep_proceedings_df['proceeding_type'] = assign_event_type(prep_proceedings_df, 'proceeding_id')

In [65]:
papers_from_journal_df['core_author'] = papers_from_journal_df['core_author'].apply(lambda x: author_to_id_map[x])
papers_from_journal_df['co_author'] = papers_from_journal_df['co_author'].apply(lambda x: [author_to_id_map.get(a) for a in x] if x is not pd.NA else pd.NA)
papers_from_journal_df['reviewer'] = papers_from_journal_df['reviewer'].apply(lambda x: [author_to_id_map.get(a) for a in x] if x is not pd.NA else pd.NA)
papers_from_journal_df['journal_id'] = papers_from_journal_df['journal'].apply(lambda x: journal_to_id_map[x])
papers_from_journal_df['volume_id'] = papers_from_journal_df['journal_id'] + '-' + papers_from_journal_df['volume'] + '-' + papers_from_journal_df['year'].astype(str)
papers_from_journal_df['keywords'] = [select_random_keywords() for _ in range(len(papers_from_journal_df))]
papers_from_journal_df['citation_id'] = add_citations_column(papers_from_journal_df)
papers_from_journal_df['abstract'] = "Abstract text"

In [66]:
journals_prep_df = journals_prep_df[journals_prep_df['id'].isin(papers_from_journal_df['journal_id'])]

In [67]:
def generate_editor(df, id_format):
    editor_names = [f"{np.random.choice(first_names)} {np.random.choice(last_names)}" for _ in range(len(df))]
    editor_ids = [id_format.format(i) for i in range(1, len(df) + 1)]
    
    editors_df = pd.DataFrame({'editor_id': editor_ids, 'editor_name': editor_names})
    
    return editor_ids, editors_df

journals_prep_df['editor_id'], editor_nodes_df = generate_editor(journals_prep_df, 'ED_{:06d}')

C:\Users\Dima\AppData\Local\Temp\ipykernel_18352\1857297096.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  journals_prep_df['editor_id'], editor_nodes_df = generate_editor(journals_prep_df, 'ED_{:06d}')


In [68]:
proceeding_prep_df

,paper_id,title,abstract,core_author,co_author,citation_id,keywords
2309,PPR-39712,Laboratory Experiments in Information Retrieva...,Abstract,10612435,<NA>,"[PPR-38537, PPR-33029, PPR-10462918, PPR-10452...","[digital humanities, quantum computing, anthro..."
15436,PPR-10458720,High-temperature-superconductor thin films at ...,Abstract,12885908,<NA>,"[PPR-10429433, PPR-10461336, PPR-10447798, PPR...","[cloud computing, agritech, ecosystem services..."
4088,PPR-51847,From scientific instrument to industrial machi...,Abstract,10654122,[10654121],"[PPR-37283, PPR-10462290, PPR-10462075, PPR-10...","[computational biology, data management, nanot..."
7580,PPR-10437867,Reasoning with Logic Programming,Abstract,10640381,[10631504],"[PPR-10456933, PPR-10458697, PPR-10459718, PPR...","[green chemistry, cancer research, wearable te..."
9199,PPR-10446776,Variation Based Dense 3D Reconstruction - Appl...,Abstract,12567759,<NA>,"[PPR-10459812, PPR-51896, PPR-10457506, PPR-33...","[digital humanities, anthropology, drug discov..."
...,...,...,...,...,...,...,...
13151,PPR-10456160,Algorithms.,Abstract,10641232,"[10600395, 11027237]","[PPR-10454987, PPR-10461523, PPR-10456927]","[space exploration, computational biology, nut..."
18075,PPR-10461667,Theorie et pratique du workflow - des processu...,Abstract,11720125,<NA>,"[PPR-10441964, PPR-10461431, PPR-10461458, PPR...","[big data, deep learning, data modeling, econo..."
5808,PPR-10430360,Contribution à l'analyse non linéaire.,Abstract,14119273,<NA>,"[PPR-10452873, PPR-10429418, PPR-10430982, PPR...","[internet of things, augmented reality, data s..."
15330,PPR-10458605,Einführung in die funktionale Programmierung.,Abstract,10634933,[10641933],"[PPR-10460319, PPR-10460820, PPR-10443697, PPR...","[machine ethics, forensic science, astrophysic..."


In [69]:
prep_proceedings_df

,proceeding_id,chairman,name,edition_date,edition_id,edition_city,paper_id,proceeding_type
471,PR-000001,CH-000001,Springer,2017-05-19,PR-000001-39,Buenos Aires,"[PPR-10455809, PPR-10457844, PPR-10429750]",conference
32328,PR-000002,CH-000002,Elsevier,2023-01-31,PR-000002-190,Kansas City,"[PPR-33763, PPR-10430602]",conference
43581,PR-000001,CH-000003,Springer,2019-05-14,PR-000001-2182,Kansas City,"[PPR-10431075, PPR-10462401]",conference
55157,PR-000001,CH-000006,Springer,2019-05-14,PR-000001-2677,Los Angeles,"[PPR-10454371, PPR-10456496, PPR-10462480]",conference
17459,PR-000001,CH-000007,Springer,2019-05-14,PR-000001-9731,Charlotte,"[PPR-10466667, PPR-10461645, PPR-10451322, PPR...",conference
...,...,...,...,...,...,...,...,...
10679,PR-000001,CH-002048,Springer,2021-09-24,PR-000001-12967,Mumbai,"[PPR-51691, PPR-52812, PPR-10438936, PPR-10469...",conference
22050,PR-000001,CH-002049,Springer,2019-05-14,PR-000001-1449,San Francisco,"[PPR-10442154, PPR-41586, PPR-10454371]",conference
18518,PR-000001,CH-002053,Springer,2023-02-09,PR-000001-319,Moscow,"[PPR-10442101, PPR-10458706, PPR-10429349, PPR...",conference
32949,PR-000001,CH-002054,Springer,2019-05-14,PR-000001-7035,Mumbai,"[PPR-10453587, PPR-10455206, PPR-10466256, PPR...",conference


In [70]:
papers_from_journal_df

,paper_id,journal,date,title,volume,year,core_author,co_author,reviewer,journal_id,volume_id,keywords,citation_id,abstract
563549,7560826,IEEE Trans. Software Eng.,2023-08-28,The Importance of Security Is in the Eye of th...,48,2022,12236503,"[10906624, 13769410]","[10616455, 10915263]",14155294,14155294-48-2022,"[data analytics, machine ethics, photovoltaics...","[9234890, 8893477, 9332981, 9636533, 8635609, ...",Abstract text
1038426,8038003,Neural Comput. Appl.,2023-12-09,Intelligent recognition of audio scene based o...,35,2023,13437468,[10911488],"[11274568, 10728217, 12254061]",14155694,14155694-35-2023,"[data querying, 3D printing, soil degradation,...","[7833008, 9529425, 9100167, 9688029, 10078928,...",Abstract text
2666482,9667672,Inf. Sci.,2023-12-22,Fuzzy forecasting based on automatic clusterin...,294,2015,12646372,[10812769],"[11889831, 10953236, 10985790]",14155947,14155947-294-2015,"[public health, bioinformatics, conservation b...","[8414870, 9400139, 9402836, 8218912, 9228440, ...",Abstract text
2937149,9939272,CoRR,2022-12-02,Context-Aware Ensemble Learning for Time Series.,abs/2211.16884,2022,12706973,"[13820516, 13368940, 11021851]","[11527890, 10624271, 10699931, 13045275]",14156859,14156859-abs/2211.16884-2022,"[data analytics, mental health, soil degradati...","[10055027, 7197837, 9267830, 9515971]",Abstract text
585438,7582715,Informatica (Slovenia),2016-02-15,Image processing and becoming conscious of its...,25,2001,13584726,<NA>,"[10975467, 13411771, 13052934, 11681170]",14155327,14155327-25-2001,"[cybersecurity, cloud computing, quantum mecha...","[7225862, 9049631, 9232920, 7226057, 8886778, ...",Abstract text
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2768304,9769539,Computing,2020-08-06,A study of regularization by monotonicity.,50,1993,13186900,[13603670],"[11949994, 11402381, 11049441]",14156815,14156815-50-1993,"[augmented reality, neuroscience, drug discove...","[7696137, 9755310, 9706374, 7533418]",Abstract text
3337057,10342015,CoRR,2020-11-18,Constrained Model-Free Reinforcement Learning ...,abs/2011.07925,2020,13315611,"[13486688, 13758252, 13755290, 10810743]","[11924402, 11485500, 11139050]",14156859,14156859-abs/2011.07925-2020,"[anthropology, renewable energy, sociology, as...","[8356384, 8330541, 9160647, 9049944, 7670936, ...",Abstract text
2485917,9486898,Electron. J. Inf. Syst. Dev. Ctries.,2022-02-04,"""Multimacy"": Performances of Intimacy on Faceb...",71,2015,13299781,<NA>,"[10828862, 11220798]",14156610,14156610-71-2015,"[environmental justice, digital humanities, ne...","[9016423, 7579255, 8743593]",Abstract text
1096196,8095773,Sensors,2023-09-30,Highly Sensitive Voltammetric Determination of...,21,2021,13554199,"[13922152, 13894157, 13891686, 13891680, 13892...",[12634690],14155711,14155711-21-2021,"[particle physics, internet of things, data pr...","[8911454, 8356384, 9775728, 7023052, 9602107, ...",Abstract text


In [71]:
papers_from_proceeding_df

,paper_id,title,abstract,year,core_author,co_author,reviewer,citation_id,keywords
2309,PPR-39712,Laboratory Experiments in Information Retrieva...,Abstract,2018,10612435,<NA>,"[Pingyi Fan, Jiunn-Tsair Chen, Joan Feigenbaum...","[PPR-38537, PPR-33029, PPR-10462918, PPR-10452...","[digital humanities, quantum computing, anthro..."
15436,PPR-10458720,High-temperature-superconductor thin films at ...,Abstract,1999,12885908,<NA>,[Hadi Meidani],"[PPR-10429433, PPR-10461336, PPR-10447798, PPR...","[cloud computing, agritech, ecosystem services..."
4088,PPR-51847,From scientific instrument to industrial machi...,Abstract,2012,10654122,[10654121],"[Patrick Jenny, Renben Shu, Michael Kusnetsov]","[PPR-37283, PPR-10462290, PPR-10462075, PPR-10...","[computational biology, data management, nanot..."
7580,PPR-10437867,Reasoning with Logic Programming,Abstract,1996,10640381,[10631504],[Johannes Fottner],"[PPR-10456933, PPR-10458697, PPR-10459718, PPR...","[green chemistry, cancer research, wearable te..."
9199,PPR-10446776,Variation Based Dense 3D Reconstruction - Appl...,Abstract,2016,12567759,<NA>,"[Juan Manuel González Nieto, Roberto Hornero, ...","[PPR-10459812, PPR-51896, PPR-10457506, PPR-33...","[digital humanities, anthropology, drug discov..."
...,...,...,...,...,...,...,...,...,...
13151,PPR-10456160,Algorithms.,Abstract,2008,10641232,"[10600395, 11027237]","[Miodrag Zivkovic, Ken Levy, Yuefan Deng]","[PPR-10454987, PPR-10461523, PPR-10456927]","[space exploration, computational biology, nut..."
18075,PPR-10461667,Theorie et pratique du workflow - des processu...,Abstract,1997,11720125,<NA>,"[Amit H. Bermano, Renate A. Schmidt, Stijn de ...","[PPR-10441964, PPR-10461431, PPR-10461458, PPR...","[big data, deep learning, data modeling, econo..."
5808,PPR-10430360,Contribution à l'analyse non linéaire.,Abstract,1976,14119273,<NA>,"[Hongwei Zhang 0001, Xiaochun Yin, Cyril Hösch...","[PPR-10452873, PPR-10429418, PPR-10430982, PPR...","[internet of things, augmented reality, data s..."
15330,PPR-10458605,Einführung in die funktionale Programmierung.,Abstract,1992,10634933,[10641933],"[Arturo Carpi, H. Idir, Jing Gao 0004, Melvyn ...","[PPR-10460319, PPR-10460820, PPR-10443697, PPR...","[machine ethics, forensic science, astrophysic..."


In [72]:
journals_prep_df

,id,name,editor_id
3,14154891,SIGMOD Rec.,ED_000001
4,14154892,World Wide Web,ED_000002
6,14154894,IEEE Trans. Educ.,ED_000003
7,14154895,Int. J. Neural Syst.,ED_000004
8,14154896,Comput. Optim. Appl.,ED_000005
...,...,...,...
1983,14156871,Int. J. High Perform. Comput. Appl.,ED_000896
1984,14156872,Digit. Creativity,ED_000897
1986,14156874,Scientometrics,ED_000898
1991,14156879,Digit. Humanit. Q.,ED_000899


In [73]:
author_nodes_df

,id,full_name
74,10598066,Yali Amit
81,10598073,Silvio Savarese
82,10598074,Sanjiv Kumar
110,10598102,M. Fatih Demirci
130,10598122,Zhi-Yong Liu 0001
...,...,...
3554048,14152040,Atish Dixit
3554190,14152182,Teresa W. Haynes
3554716,14152708,Dinny H. de Bakker
3555292,14153284,Craig S. Levin


In [74]:
chairman_nodes_df

,person_id,person_name
0,CH-000001,Andrzej Kwiecien
1,CH-000002,Fausto Spoto
2,CH-000003,Franco Zambonelli
3,CH-000004,Annamaria Recupero
4,CH-000005,Fumihiro Kumeno
...,...,...
2054,CH-002055,Aggelos Biboudis
2055,CH-002056,Claudio Alvarez-Gómez
2056,CH-002057,Daniel Panario
2057,CH-002058,Hajime Asama


In [75]:
editor_nodes_df

,editor_id,editor_name
0,ED_000001,Alex Davis
1,ED_000002,Dana Martinez
2,ED_000003,Drew Williams
3,ED_000004,Jordan Rodriguez
4,ED_000005,Jamie Moore
...,...,...
895,ED_000896,Skyler Thomas
896,ED_000897,Drew Smith
897,ED_000898,Morgan Williams
898,ED_000899,Kelly Jackson


# Neo4j import folder path

In [76]:
neo4j_import_folder_path = 'C:\\Users\\Dima\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-966f24ac-8bf3-4b01-a39c-321fcafcdbb2\\import'

# Slice nodes #

In [77]:
papers_from_journal_nodes_df = papers_from_journal_df[['paper_id', 'title', 'keywords', 'abstract']].drop_duplicates(['paper_id', 'title', 'abstract'])
papers_from_proceeding_nodes_df = papers_from_proceeding_df[['paper_id', 'title', 'keywords', 'abstract']].drop_duplicates(['paper_id', 'title', 'abstract'])

journal_nodes_df = journals_prep_df[['id', 'name']].drop_duplicates(['id', 'name'])
proceeding_nodes_df = prep_proceedings_df[['proceeding_id', 'name']].drop_duplicates(['proceeding_id', 'name'])

volume_nodes_df = papers_from_journal_df[['volume_id', 'year']].drop_duplicates(['volume_id'])
edition_nodes_df = prep_proceedings_df[['edition_id', 'edition_city', 'edition_date']].drop_duplicates(['edition_id', 'edition_date'])

# Save nodes

In [78]:
author_nodes_df.to_csv(f'{neo4j_import_folder_path}/author_nodes.csv', index=False)
editor_nodes_df.to_csv(f'{neo4j_import_folder_path}/editor_nodes.csv', index=False)
proceeding_nodes_df.to_csv(f'{neo4j_import_folder_path}/proceeding_nodes.csv', index=False)
number_of_editors = len(proceeding_nodes_df)
chairman_nodes_df = chairman_nodes_df.iloc[:number_of_editors]
chairman_nodes_df.to_csv(f'{neo4j_import_folder_path}/chairman_nodes.csv', index=False)

In [79]:
print(len(proceeding_nodes_df))

43


In [80]:
papers_from_journal_nodes_df.to_csv(f'{neo4j_import_folder_path}/papers_from_journal_nodes.csv', index=False)
papers_from_proceeding_nodes_df.to_csv(f'{neo4j_import_folder_path}/papers_from_proceeding_nodes.csv', index=False)

In [81]:
journal_nodes_df.to_csv(f'{neo4j_import_folder_path}/journal_nodes.csv', index=False)

In [82]:
volume_nodes_df.to_csv(f'{neo4j_import_folder_path}/volume_nodes.csv', index=False)
edition_nodes_df.to_csv(f'{neo4j_import_folder_path}/edition_nodes.csv', index=False)

# Slice relations #

In [83]:
paper_CITES_paper_journal_df = papers_from_journal_df[['paper_id', 'citation_id']].explode('citation_id').dropna()
paper_CITES_paper_proceeding_df = papers_from_proceeding_df[['paper_id', 'citation_id']].explode('citation_id').dropna()

paper_PUBLISHED_IN_WORKSHOP_edition_proceeding_df = prep_proceedings_df[prep_proceedings_df['proceeding_type'] == 'workshop'][['paper_id', 'edition_id']].explode('paper_id').drop_duplicates(['paper_id', 'edition_id'])
paper_PUBLISHED_IN_CONFERENCE_edition_proceeding_df = prep_proceedings_df[prep_proceedings_df['proceeding_type'] == 'conference'][['paper_id', 'edition_id']].explode('paper_id').drop_duplicates(['paper_id', 'edition_id'])

paper_PUBLISHED_IN_VOLUME_journal_df = papers_from_journal_df[['paper_id', 'volume_id']].drop_duplicates(['paper_id', 'volume_id'])

author_IS_CORE_AUTHOR_paper_journal_df = papers_from_journal_df[['paper_id', 'core_author']]
author_IS_CORE_AUTHOR_paper_proceeding_df = papers_from_proceeding_df[['paper_id', 'core_author']]

author_IS_CO_AUTHOR_paper_journal_df = papers_from_journal_df[['paper_id', 'co_author']].explode('co_author').dropna()
author_IS_CO_AUTHOR_paper_proceeding_df = papers_from_proceeding_df[['paper_id', 'co_author']].explode('co_author').dropna()

author_REVIEWS_paper_journal_df = papers_from_journal_df[['paper_id', 'reviewer']].explode('reviewer')
author_REVIEWS_paper_proceeding_df = papers_from_proceeding_df[['paper_id', 'reviewer']].explode('reviewer')

proceeding_CONTAINS_WORKSHOP_edition_df = prep_proceedings_df[prep_proceedings_df['proceeding_type'] == 'workshop'][['proceeding_id', 'edition_id']].drop_duplicates(['proceeding_id', 'edition_id'])
proceeding_CONTAINS_CONFERENCE_edition_df = prep_proceedings_df[prep_proceedings_df['proceeding_type'] == 'conference'][['proceeding_id', 'edition_id']].drop_duplicates(['proceeding_id', 'edition_id'])
journal_CONTAINS_VOLUME_volume_df = papers_from_journal_df[['journal_id', 'volume_id']].drop_duplicates(['journal_id', 'volume_id'])

journal_HAS_EDITOR_person_df = journals_prep_df[['id', 'editor_id']].drop_duplicates(['id', 'editor_id'])
proceeding_HAS_WORKSHOP_CHAIRMAN_person_df = prep_proceedings_df[prep_proceedings_df['proceeding_type'] == 'workshop'][['proceeding_id', 'chairman']].drop_duplicates(['proceeding_id', 'chairman'])
proceeding_HAS_CONFERENCE_CHAIRMAN_person_df = prep_proceedings_df[prep_proceedings_df['proceeding_type'] == 'conference'][['proceeding_id', 'chairman']].drop_duplicates(['proceeding_id', 'chairman'])

In [84]:
paper_PUBLISHED_IN_WORKSHOP_edition_proceeding_df

,paper_id,edition_id
46127,PPR-10430531,PR-000004-7341
46127,PPR-10455989,PR-000004-7341
44801,PPR-10430513,PR-000005-706
44801,PPR-10455750,PR-000005-706
44801,PPR-10462261,PR-000005-706
...,...,...
36845,PPR-40107,PR-000005-1372
36845,PPR-10448685,PR-000005-1372
36845,PPR-10437690,PR-000005-1372
36845,PPR-10462855,PR-000005-1372


In [86]:
paper_CITES_paper_journal_df.to_csv(f'{neo4j_import_folder_path}/paper_CITES_paper_journal.csv', index=False)
paper_CITES_paper_proceeding_df.to_csv(f'{neo4j_import_folder_path}/paper_CITES_paper_proceeding.csv', index=False)
paper_PUBLISHED_IN_WORKSHOP_edition_proceeding_df.to_csv(f'{neo4j_import_folder_path}/paper_PUBLISHED_IN_WORKSHOP_edition_proceeding.csv', index=False)
paper_PUBLISHED_IN_CONFERENCE_edition_proceeding_df.to_csv(f'{neo4j_import_folder_path}/paper_PUBLISHED_IN_CONFERENCE_edition_proceeding.csv', index=False)
paper_PUBLISHED_IN_VOLUME_journal_df.to_csv(f'{neo4j_import_folder_path}/paper_PUBLISHED_IN_VOLUME_journal.csv', index=False)
author_IS_CORE_AUTHOR_paper_journal_df.to_csv(f'{neo4j_import_folder_path}/author_IS_CORE_AUTHOR_paper_journal.csv', index=False)
author_IS_CORE_AUTHOR_paper_proceeding_df.to_csv(f'{neo4j_import_folder_path}/author_IS_CORE_AUTHOR_paper_proceeding.csv', index=False)
author_IS_CO_AUTHOR_paper_journal_df.to_csv(f'{neo4j_import_folder_path}/author_IS_CO_AUTHOR_paper_journal.csv', index=False)
author_IS_CO_AUTHOR_paper_proceeding_df.to_csv(f'{neo4j_import_folder_path}/author_IS_CO_AUTHOR_paper_proceeding.csv', index=False)
author_REVIEWS_paper_journal_df.to_csv(f'{neo4j_import_folder_path}/author_REVIEWS_paper_journal.csv', index=False)
author_REVIEWS_paper_proceeding_df.to_csv(f'{neo4j_import_folder_path}/author_REVIEWS_paper_proceeding.csv', index=False)
proceeding_CONTAINS_WORKSHOP_edition_df.to_csv(f'{neo4j_import_folder_path}/proceeding_CONTAINS_WORKSHOP_edition.csv', index=False)
proceeding_CONTAINS_CONFERENCE_edition_df.to_csv(f'{neo4j_import_folder_path}/proceeding_CONTAINS_CONFERENCE_edition.csv', index=False)
journal_CONTAINS_VOLUME_volume_df.to_csv(f'{neo4j_import_folder_path}/journal_CONTAINS_VOLUME_volume.csv', index=False)
journal_HAS_EDITOR_person_df.to_csv(f'{neo4j_import_folder_path}/journal_HAS_EDITOR_person.csv', index=False)
proceeding_HAS_WORKSHOP_CHAIRMAN_person_df.to_csv(f'{neo4j_import_folder_path}/proceeding_HAS_WORKSHOP_CHAIRMAN_person.csv', index=False)
proceeding_HAS_CONFERENCE_CHAIRMAN_person_df.to_csv(f'{neo4j_import_folder_path}/proceeding_HAS_CONFERENCE_CHAIRMAN_person.csv', index=False)